In [ ]:
import polars as pl

In [ ]:
import json

In [ ]:
import numpy as np
import copy

In [ ]:
with open('I-24MOTION_2022-11-17_08-10-00.json', 'r') as f:
    data = json.load(f)

In [ ]:
# data = copy.deepcopy(_data)

In [ ]:
datum = data[0]

In [ ]:
datum['trajectory_id']

In [ ]:
for d in data[:10]:
    print(d['direction'])

In [ ]:
for k, v in datum.items():
    print(k, type(v))

In [ ]:
type_map = {}
for datum in data:
    for k, v in datum.items():
        if k not in type_map:
            type_map[k] = set()
        type_map[k].add(type(v))

In [ ]:
# type_map = {}
# for datum in data:
#     if isinstance(datum['total_fuel_economy_flat_road_mpg'], list):
#         print(datum['total_fuel_economy_flat_road_mpg'])

In [ ]:
pl.from_records([type_map])

In [ ]:
mixed_type = []
for k, v in type_map.items():
    if len(v) > 1:
        print(v, k)
        mixed_type.append(k)
mixed_type

In [ ]:
list_type = {}
for datum in data:
    for k, v in datum.items():
        if isinstance(v, list):
            if k not in list_type:
                list_type[k] = set()
            if not isinstance(v, list):
                list_type[k].add(type(v))
            elif len(v) == 0:
                list_type[k].add('empty')
            else:
                if any(isinstance(vv, float) for vv in v):
                    list_type[k].add('float')
                # if np.array(v).dtype == np.dtype('O'):
                #     print(v)
                elif any(isinstance(vv, int) for vv in v):
                    list_type[k].add('int')
                else:
                    assert False
                    # list_type[k].add('unknown')
            # if np.array(v).dtype == np.dtype('O'):
            #     print(v)

In [ ]:
for k, v in list_type.items():
    print(k, v)

In [ ]:
object_dtypes = []
float_dtypes = []
for k, v in list_type.items():
    if len(v) > 1 and np.dtype('O') in v:
        print(k, v)
        object_dtypes.append(k)
    elif 'float' in v:
        print(k, v)
        float_dtypes.append(k)
    # elif len(v) == 1 and np.dtype('float64') in v:
    #     float_dtypes.append(k)

In [ ]:
np.array([1,2,3]).astype(np.dtype('O'))

In [ ]:
for datum in data:
    aid = datum['upstream_av_id']
    # if (isinstance(aid, list) and any(isinstance(a, float) for a in aid)) or (aid is None):
    #     print(aid)
    for t in mixed_type:
        d = datum[t]
        if isinstance(d, list) or d is None:
            datum[t] = None
        else:
            datum[t] = float(d)
    for t in float_dtypes:
        d = datum[t]
        if d is None:
            continue
        if isinstance(d, float):
            d = [d]
        assert isinstance(d, list), type(d)
        datum[t] = [*map(lambda x: float(x) if x is not None else None, d)]
    # for t in object_dtypes:
    #     d = datum[t]
    #     datum[t] = [(None if dd is None else float(dd)) for dd in d]

In [ ]:
data_dict = {}
for datum in data:
    for k in type_map:
        if k not in data_dict:
            data_dict[k] = []
        data_dict[k].append(datum[k])

In [ ]:
data_dict.keys()

In [ ]:
for k, v in data_dict.items():
    try:
        pl.DataFrame({k: v})
    except TypeError as e:
        print(e)
        s = set()
        for vv in v:
            s.add(str(np.array(vv).dtype))
        print(s)
        print(k)
        print(np.array(v).shape)
        print([])
        break

In [ ]:
df = pl.from_records(data)
df

In [ ]:
import pickle
with open('./data.pkl', 'wb') as f:
    pickle.dump(df, f)